# [How to pass tool outputs to chat models](https://python.langchain.com/docs/how_to/tool_results_pass_to_model/)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) ->int:
    """add a and b."""
    return a + b

@tool
def multiply(a: int, b: int) ->int:
    """multiply a and b."""
    return a * b

tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

Now, let's get the model to call a tool. We'll add it to a list of messages that we'll treat as conversation history:

In [ ]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

# print(ai_msg)
print(ai_msg.tool_calls)

messages.append(ai_msg)
print(messages)

Next let's invoke the tool functions using the args the model populated!

Conveniently, if we invoke a LangChain `Tool` with a `ToolCall`, we'll automatically get back a `ToolMessage` that can be fed back to the model:



In [ ]:
tool_list = {"add": add, "multiply": multiply}

for tool_call in ai_msg.tool_calls:
    # selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    selected_tool = tool_list[tool_call["name"].lower()]

    print(selected_tool)

    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

In [ ]:
res = llm_with_tools.invoke(messages)
print(res)